In [ ]:
#| default_exp embed_images

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export

from pathlib import Path
from PIL import Image
import numpy as np
import torch
from tqdm import tqdm
from loguru import logger
from transformers import pipeline
import daft

from bedmap.config import Cfg
from bedmap.step import step

In [ ]:
#| export

BATCH_SIZE = 4

In [ ]:
#| export

def images_from_paths(pathlist):
    return (Image.open(p.as_posix()).convert("RGB").copy() for p in pathlist)

In [ ]:
#| export

def embed_images(imagepaths : list[Path],
                 model_name : str = "timm/vit_small_patch14_reg4_dinov2.lvd142m",
                 batch_size : int = 4
                 ) -> list[np.array]:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pipe = pipeline(task="image-feature-extraction",
                    model=model_name, device=device, pool=True, use_fast=True)

    # logger.info("Starting embedding pipeline.")
    embeddings = []

    for out in tqdm(pipe(images_from_paths(imagepaths), batch_size=batch_size),
                    total=len(imagepaths)//batch_size):
        embeddings += out

    # logger.info("Done with embedding pipeline.")

    return np.array(embeddings)

In [ ]:
#| export

@step(requires=["img_path"], provides=["embeddings"])
def create_embeddings_col(df: daft.DataFrame, model_name: str, batch_size: int) -> daft.DataFrame:
    """
    Embed images for a given dataframe.
    """
    ## Surely there's a cleaner way to get the paths out

    paths = [Path(i["img_path"].lstrip("file:/")) for i in df.select("img_path").to_pylist()]
    embeds = embed_images(paths, model_name=model_name, batch_size=batch_size)
    embeds_type = daft.DataType.embedding(daft.DataType.float32(), embeds.shape[-1])
    embeds_series = daft.Series.from_numpy(embeds).cast(embeds_type)

    return df.with_column("embeddings", daft.lit(embeds_series))

In [ ]:
#| hide

from bedmap.prepare_images import df_images_from_pattern
from bedmap.config import Cfg

cfg = Cfg()
TEST_DIR = "../tests/test-data/smithsonian_butterflies_10/jpgs"

df = df_images_from_pattern(TEST_DIR)
df = create_embeddings_col(df, model_name=cfg.model_name, batch_size=BATCH_SIZE)

🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 Project: 00:00 

Device set to use cpu
12it [00:13,  1.11s/it]                      


In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()